In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, IntSlider, FloatSlider # To są nasze suwaki!

# 1. GENERUJEMY DANE (To samo co wcześniej, żeby było na czym pracować)
np.random.seed(42)
ndvi_data = np.clip(np.random.normal(0.78, 0.05, 10000), 0.1, 0.95)
df = pd.DataFrame({'NDVI': ndvi_data})

# Funkcja plonowania (z karą za przenawożenie)
def oblicz_plon(ndvi, dawka_azotu):
    potencjal = ndvi * 12
    if dawka_azotu <= 190:
        wsp = min(dawka_azotu / 160, 1.0)
    else:
        # Kara: każde 10kg nadmiaru zabiera 1% plonu
        nadmiar = dawka_azotu - 190
        wsp = 1.0 - (nadmiar * 0.001)
    return max(potencjal * wsp + np.random.normal(0, 0.2), 0)

# 2. DEFINIUJEMY SCENARIUSZE (Stałe dawki nawozu)
# Tradycyjny sypie 220 kg (błąd), Precyzyjny dobiera dawkę (140-170 kg)
df['N_Tradycyjny'] = 220
df['Plon_Tradycyjny'] = df.apply(lambda x: oblicz_plon(x['NDVI'], 220), axis=1)

def dobierz_dawke(ndvi):
    if ndvi < 0.70: return 140
    elif ndvi > 0.82: return 170
    else: return 160

df['N_Precyzyjny'] = df['NDVI'].apply(dobierz_dawke)
df['Plon_Precyzyjny'] = df.apply(lambda x: oblicz_plon(x['NDVI'], x['N_Precyzyjny']), axis=1)


# 3. INTERAKTYWNA APLIKACJA (Tu dzieje się magia!)
# Ta funkcja uruchomi się za każdym razem, gdy ruszysz suwakiem

def symulator_biznesowy(cena_pszenicy, cena_nawozu):

    # Przeliczamy zyski na żywo w oparciu o suwaki
    zysk_A = (df['Plon_Tradycyjny'] * cena_pszenicy) - (df['N_Tradycyjny'] * cena_nawozu)
    zysk_B = (df['Plon_Precyzyjny'] * cena_pszenicy) - (df['N_Precyzyjny'] * cena_nawozu)

    srednia_A = zysk_A.mean()
    srednia_B = zysk_B.mean()
    roznica = srednia_B - srednia_A

    # Rysujemy wykres
    plt.figure(figsize=(10, 6))

    # Kolory: Czerwony dla tradycyjnego, Zielony dla VRA
    bars = plt.bar(['Tradycyjna', 'Precyzyjna (VRA)'], [srednia_A, srednia_B],
                   color=['#e74c3c', '#2ecc71'], alpha=0.9)

    # Dodajemy linię poziomu opłacalności (np. czy zarobiliśmy cokolwiek?)
    plt.axhline(0, color='black', linewidth=1)

    # Napisy na słupkach
    plt.text(0, srednia_A + 100, f'{srednia_A:.0f} zł', ha='center', fontsize=14, weight='bold')
    plt.text(1, srednia_B + 100, f'{srednia_B:.0f} zł', ha='center', fontsize=14, weight='bold')

    # Tytuł dynamiczny
    plt.title(f'SYMULACJA ZYSKU (Różnica: +{roznica:.0f} zł/ha)', fontsize=16)
    plt.ylabel('Zysk operacyjny (zł/ha)')
    plt.ylim(0, max(srednia_B, srednia_A) * 1.2) # Skalowanie wykresu
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.show()

# 4. ODPALAMY PANEL STEROWANIA
print("🎛️ TWOJE CENTRUM DOWODZENIA (Przesuwaj suwaki!):")
interact(symulator_biznesowy,
         cena_pszenicy = IntSlider(min=600, max=1400, step=50, value=900, description='Cena Zboża (zł/t)'),
         cena_nawozu = FloatSlider(min=2.0, max=10.0, step=0.5, value=5.0, description='Cena Azotu (zł/kg)'));